<a href="https://colab.research.google.com/github/rroxy08/CE888/blob/master/assignment/kliep/ass2_covariate_ex4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
##########################ieee-fraud-detection###########################

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random as rd
import os
from sklearn.metrics import accuracy_score
from sklearn.ensemble import ExtraTreesClassifier

from sklearn import preprocessing
from sklearn.neighbors import KernelDensity
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from scipy.ndimage.filters import gaussian_filter
from scipy.stats import norm
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
#drive.mount('/content/gdrive')
os.environ['KAGGLE_USERNAME'] = "khaledabegum"
os.environ['KAGGLE_KEY'] = "3ab75f2bb1de78f9addea9a3f5079ec9"


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
!rm -r pykliep
!git clone https://github.com/srome/pykliep/
#!git clone https://github.com/srome/pykliep/blob/master/pykliep.py

#!git clone --recurse-submodules https://github.com/srome/pykliep.git
import sys
sys.path.append('/content/khaleda/')


Cloning into 'pykliep'...
remote: Enumerating objects: 15, done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (15/15), done.


In [0]:

import pykliep as pk


In [0]:
#open('mylib.py','wb')
#import mylib
from pykliep import pykliep

In [7]:
!pip install densratio
from densratio import densratio

In [8]:
#!kaggle competitions download --force -c sberbank-russian-housing-market
#!kaggle competitions download --force -c sberbank-russian-housing-market
!kaggle competitions download --force -c microsoft-malware-prediction


100% 134M/134M [00:00<00:00, 116MB/s]
100% 134M/134M [00:01<00:00, 139MB/s]
100% 669M/672M [00:03<00:00, 228MB/s]
100% 672M/672M [00:03<00:00, 206MB/s]
 99% 757M/768M [00:05<00:00, 150MB/s]
100% 768M/768M [00:05<00:00, 146MB/s]


In [9]:
from zipfile import ZipFile

file_names = ["/content/train.csv.zip","/content/test.csv.zip"]

for file_name in file_names:
  with ZipFile(file_name, 'r') as zip:
    zip.extractall()
    print('done')

done
done


In [10]:
root_path1 = '/content/train.csv' 
root_path2 = '/content/test.csv' 
#df_train=pd.read_csv(root_path1)
#df_test=pd.read_csv(root_path2)

df_train=pd.read_csv(root_path1)
df_test=pd.read_csv(root_path2)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
#df_train=df_train.sample(20000)
#df_test=df_test.sample(20000)

df_tr=df_train.sample(20000)
df_ts=df_test.sample(20000)

df_train.count
df_train.head()
#df_tr=df_train.copy()
#df_ts=df_test.copy()


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,...,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,29,128035.0,18.0,35.0,171,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Desktop,Windows.Desktop,2668.0,9124.0,4.0,5.0,...,HDD,299451.0,0,4096.0,Desktop,18.9,1440.0,900.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Professional,PROFESSIONAL,UUPUpgrade,26.0,119,UNKNOWN,0,IS_GENUINE,Retail,NaN,0.0,Retail,NaN,628.0,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,93,1482.0,18.0,119.0,64,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Notebook,Windows.Desktop,2668.0,91656.0,4.0,5.0,...,HDD,102385.0,0,4096.0,Notebook,13.9,1366.0,768.0,Mobile,NaN,1.000000e+00,10.0.17134.1,amd64,rs4_release,17134,1,Professional,PROFESSIONAL,IBSClean,8.0,31,UNKNOWN,0,OFFLINE,Retail,NaN,0.0,NOT_SET,NaN,628.0,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,86,153579.0,18.0,64.0,49,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,RequireAdmin,1.0,1.0,Desktop,Windows.Desktop,4909.0,317701.0,4.0,5.0,...,SSD,113907.0,0,4096.0,Desktop,21.5,1920.0,1080.0,Desktop,NaN,4.294967e+09,10.0.17134.165,amd64,rs4_release,17134,165,Core,CORE,UUPUpgrade,7.0,30,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,142.0,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,88,20710.0,NaN,117.0,115,windows10,x64,10.0.0.0,17134,256,rs4,17134.1.amd64fre.rs4_release.180410-1804,Pro,1.0,0,NaN,0.0,137.0,ExistsNotSet,1.0,1.0,Desktop,Windows.Desktop,1443.0,275890.0,4.0,5.0,...,UNKNOWN,227116.0,0,4096.0,MiniTower,18.5,1366.0,768.0,Desktop,NaN,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Professional,PROFESSIONAL,UUPUpgrade,17.0,64,FullAuto,0,IS_GENUINE,OEM:NONSLP,NaN,0.0,Retail,NaN,355.0,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417e

In [13]:
## Label encoding for string/categorical values
number = LabelEncoder()

#preprocessing of data, fillup the NAN value. Since 'year' data types are float so mode is used to fill NAN samples.

for i in df_tr.columns:
    if (df_tr[i].dtype == 'object'):
      df_tr[i] = df_tr[i].fillna(df_tr[i].mode().iloc[0])
    if (df_tr[i].dtype == 'int'or df_tr[i].dtype == 'float'):
      df_tr[i] = df_tr[i].fillna(np.mean(df_tr[i]))

for i in df_ts.columns:
    if (df_ts[i].dtype == 'object'):
      df_ts[i] = df_ts[i].fillna(df_ts[i].mode().iloc[0])
    if (df_ts[i].dtype == 'int' or df_ts[i].dtype == 'float'):
      df_ts[i] = df_ts[i].fillna(np.mean(df_ts[i]))

for i in df_tr.columns:
    if (df_tr[i].dtype == 'object'):
      df_tr[i] = number.fit_transform(df_tr[i].astype('str'))
      df_tr[i] = df_tr[i].astype('object')

for i in df_ts.columns:
    if (df_ts[i].dtype == 'object'):
      df_ts[i] = number.fit_transform(df_ts[i].astype('str'))
      df_ts[i] = df_ts[i].astype('object')

df_tr.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,...,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
1312294,2963,1,33,28,1867,0,7.0,0,1651.502132,53447.0,1.0,1.0,1,86,106830.0,24.866839,32.0,50,0,1,0,17134,256,4,156,6,1.0,0,0,0.0,137.000000,6,1.000000,1.0,2,0,1980.0,205313.0,4.0,5.0,...,3,79914.0,0,8192.0,4,18.5,1366.0,768.0,1,8,4.294967e+09,188,0,5,17134,228,11,11,6,8.0,31,2,0,1,2,0.0,0.0,3,0.000137,168.0,33778.0,0,0.0,0.0,0,0,0.0,1.0,3.0,0
3114333,6984,0,32,9,1371,0,7.0,0,1651.502132,29199.0,1.0,1.0,0,141,90129.0,27.000000,167.0,227,2,2,2,7601,256,7,213,6,1.0,0,0,0.0,335.000000,6,0.979285,1.0,2,0,1443.0,275889.0,2.0,5.0,...,3,150546.0,0,2048.0,17,18.9,1440.0,900.0,4,8,1.136708e+09,46,2,12,10586,318,11,11,7,9.0,34,2,0,1,2,0.0,0.0,3,0.000000,355.0,19889.0,0,0.0,0.0,0,0,0.0,0.0,10.0,1
3244631,7284,1,33,13,1624,0,7.0,0,1651.502132,7945.0,2.0,1.0,1,188,129061.0,27.000000,78.0,223,0,2,0,16299,768,3,151,4,1.0,0,0,0.0,117.000000,6,1.000000,1.0,7,0,2206.0,242095.0,2.0,5.0,...,0,288039.0,0,2048.0,13,10.0,1024.0,600.0,3,6,0.000000e+00,168,2,2,16299,492,1,1,8,9.0,34,3,0,1,2,0.0,0.0,3,0.000000,500.0,33116.0,0,0.0,0.0,0,0,0.0,0.0,10.0,0
5204616,11737,1,32,28,1551,0,7.0,0,1651.502132,53447.0,1.0,1.0,1,195,102241.0,18.000000,277.0,75,0,1,0,17134,256,4,156,6,1.0,0,0,0.0,137.000000,6,1.000000,1.0,7,0,525.0,257590.0,4.0,5.0,...,0,714788.0,0,8192.0,13,13.9,1366.0,768.0,3,8,0.000000e+00,194,0,5,17134,48,11,11,6,8.0,31,5,0,1,2,0.0,0.0,4,0.000137,142.0,9627.0,0,0.0,0.0,0,0,0.0,1.0,1.0,0
7690488,17297,1,32,28,1272,0,7.0,0,1651.502132,53447.0,1.0,1.0,1,173,2018.0,18.000000,277.0,75,0,1,0,17134,256,4,156,6,1.0,0,0,0.0,126.451032,2,1.000000,1.0,7,0,4730.0,302952.0,8.0,5.0,...,0,118533.0,0,4096.0,13,15.5,1366.0,768.0,3,8,0.000000e+00,185,0,5,17134,165,11,11,6,8.0,31,2,0,0,2,0.0,0.0,3,0.000000,556.0,13908.0,0,0.0,0.0,0,0,0.0,0.0,11.0,1


In [0]:
df_ts.head()
#df_tr['price_doc'].value_counts()


In [11]:
df_tr.shape
df_ts.shape

(80000, 82)

In [0]:
training=df_tr.sample(20000)
testing=df_ts.sample(20000)
bin_val=10
x = np.linspace( 0, 10, 20000)
for col in training.columns[2:20]: 
#for i in range(df_ts.size):
  #col='var_'+str(i)  
  #print(col)
  hist_1, bins = np.histogram(training[col], bins=bin_val,density=True) 
  hist_2, bins = np.histogram(testing[col], bins=bin_val,density=True)
  
  hist_r=hist_1/np.sum(hist_1)
  hist_s=hist_2/np.sum(hist_2)

  
  fig = plt.figure(figsize=(10,5))
  ax1 = fig.add_subplot(121) 
  ax1.hist(training[col], bins=10, color='blue',label='train data',alpha=0.2) 
  ax1.hist(testing[col], bins=10, color='red',label='test data', alpha=0.2) 

    
  ax1.set_title('column- '+col) 
  ax1.legend()
  


In [0]:
df_tr['HasDetections'].value_counts()

0    4462591
1    4458892
Name: HasDetections, dtype: int64

In [14]:
if('HasDetections' in df_tr.columns):
  del df_tr['HasDetections']

df_tr['is_test'] = range(1, len(df_tr) + 1)
df_ts['is_test'] = range(1, len(df_ts) + 1)

df_tr['is_test'] = 0
df_ts['is_test'] = 1

#if('ID_code' in df_merge.columns):
#  del df_merge['ID_code']


df_merge=pd.concat([df_tr,df_ts], sort=False)
df_merge.head


<bound method NDFrame.head of         MachineIdentifier ProductName  ... Wdft_RegionIdentifier is_test
1312294              2963           1  ...                   3.0       0
3114333              6984           0  ...                  10.0       0
3244631              7284           1  ...                  10.0       0
5204616             11737           1  ...                   1.0       0
7690488             17297           1  ...                  11.0       0
...                   ...         ...  ...                   ...     ...
6731444             17120           1  ...                  15.0       1
7002304             17781           1  ...                  13.0       1
3775781              9624           1  ...                   1.0       1
1169852              3034           1  ...                  15.0       1
7663444             19482           1  ...                   1.0       1

[40000 rows x 83 columns]>

In [0]:
Y=df_merge['is_test']

if('is_test' in df_merge.columns):
  del df_merge['is_test']
  
if('MachineIdentifier' in df_merge.columns):
  del df_merge['MachineIdentifier']
x_train, x_test, y_train, y_test = train_test_split(df_merge,Y, test_size=0.2)

In [0]:
#####################   PRUNNING   ###########################################
'''
max_depth = []
acc_gini = []
acc_entropy = []
for i in range(1,30):
 dtree = DecisionTreeClassifier(criterion = "gini", splitter = 'random', max_leaf_nodes = 10, min_samples_leaf = 5, max_depth= i)
 
 dtree.fit(x_train, y_train)
 pred = dtree.predict(x_test)
 acc_gini.append(accuracy_score(y_test, pred))
 ####
 dtree = DecisionTreeClassifier(criterion = "entropy", splitter = 'random', max_leaf_nodes = 10, min_samples_leaf = 5, max_depth= i)
 dtree.fit(x_train, y_train)
 pred = dtree.predict(x_test)
 acc_entropy.append(accuracy_score(y_test, pred))
 ####
 max_depth.append(i)

d = pd.DataFrame({'acc_gini':pd.Series(acc_gini), 
 'acc_entropy':pd.Series(acc_entropy),
 'max_depth':pd.Series(max_depth)})
# visualizing changes in parameters
plt.plot('max_depth','acc_gini', data=d, label='gini')
plt.plot('max_depth','acc_entropy', data=d, label='entropy')
plt.xlabel('max_depth')
plt.ylabel('accuracy')
plt.legend()
'''

In [0]:
####################### K_FOLD ##########################
'''
#df_merge=df_merge.sample(300000)
kf = KFold(n_splits =5, shuffle = True)
#clf = RandomForestClassifier(n_estimators=50, max_depth=2, random_state=0, max_features='sqrt', min_samples_leaf=10)
rf_reg= DecisionTreeClassifier(criterion = "gini", splitter = 'random', max_leaf_nodes = 10, min_samples_leaf = 5, max_depth= 2) #BaggingClassifier(base_estimator= clf)
plt.figure()
scores = []
for i in range(5):
    result = next(kf.split(df_merge), None)
    x_train = df_merge.iloc[result[0]]
    x_test = df_merge.iloc[result[1]]
    y_train = Y.iloc[result[0]]
    y_test = Y.iloc[result[1]]
    model = rf_reg.fit(x_train,y_train)
    predictions = rf_reg.predict(x_test)
    #cnf_matrix = confusion_matrix(y_test, predictions)
    #plot_confusion_matrix( rf_reg, x_test, predictions,normalize = True)
    scores.append(model.score(x_test,y_test))
print('Scores from each Iteration: ', scores)
print('Average K-Fold Score :' , np.mean(scores))
'''

In [16]:
#clf=DecisionTreeClassifier(max_depth=5)
clf= DecisionTreeClassifier(criterion = "gini", splitter = 'random', max_leaf_nodes = 10, min_samples_leaf = 5, max_depth= 2)
#clf=RandomForestClassifier(max_depth=2, random_state=0)
# Building the model
clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
print("ROC-AUC: ",accuracy_score(y_test, y_pred))

results = cross_val_score(clf, x_test, y_test, cv=10)
print("Score:", (results.mean()))


ROC-AUC:  0.99325
Score: 0.955875


In [0]:
arr_train=np.array(x_train)
arr_test=np.array(x_test)

# Estimate the density ratio w
dratio =densratio(arr_train,arr_test, kernel_num=10)

density_ratio = dratio.compute_density_ratio(arr_train)
print(density_ratio)


In [0]:
#clf= DecisionTreeClassifier(criterion = "gini", splitter = 'random', max_leaf_nodes = 10, min_samples_leaf = 5, max_depth= 2)
#clf=RandomForestClassifier(max_depth=2, random_state=0)
# Building the model
clf.fit(x_train, y_train, sample_weight=density_ratio)
y_pred=clf.predict(x_test)
print("ROC-AUC: ",accuracy_score(y_test, y_pred))

results = cross_val_score(clf, x_test, y_test, cv=10)
print("Score:", (results.mean()))    

ROC-AUC:  0.9475
Score: 0.84275


In [21]:
#from pykliep import DensityRatioEstimator 
arr_train=x_train.values.astype(float)
arr_test=x_test.values.astype(float)


kliep = pykliep.DensityRatioEstimator()
kliep.fit(arr_train, arr_test) # keyword arguments are X_train and X_test
#weights = kliep.predict(x_train)
'''
clf.fit(x_train, x_train, sample_weight=weights) # Train using the sample weights!
y_pred=clf.predict(x_test)
print("ROC-AUC: ",accuracy_score(y_test, y_pred))

results = cross_val_score(clf, x_test, y_test, cv=10)
print("Score:", (results.mean()))
''' 

/content/pykliep/pykliep.py:167: RuntimeWarning: divide by zero encountered in true_divide
  out += epsilon*np.dot(np.transpose(A),1./np.dot(A,out))


AttributeError: ignored